In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import functions as pfunc
from pyspark.sql import SQLContext
from pyspark.sql import Window, types

sc = pyspark.SparkContext(appName="phishtank-AUTOML-BestModel-Discovery")
sqlContext = SQLContext(sc)

import h2o
from h2o.automl import H2OAutoML
import subprocess
subprocess.run('unset http_proxy', shell=True)
h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_172"; Java(TM) SE Runtime Environment (build 1.8.0_172-b11); Java HotSpot(TM) 64-Bit Server VM (build 25.172-b11, mixed mode)
  Starting server from /opt/cloudera/parcels/Anaconda-5.1.0.1/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmphn_p909m
  JVM stdout: /tmp/tmphn_p909m/h2o_siemanalyst_started_from_python.out
  JVM stderr: /tmp/tmphn_p909m/h2o_siemanalyst_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.1
H2O cluster version age:,16 days
H2O cluster name:,H2O_from_python_siemanalyst_woo8vh
H2O cluster total nodes:,1
H2O cluster free memory:,5.176 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"accepting new members, healthy"


In [ ]:
import pandas as pd
import numpy as np
from pandas import DataFrame

from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from pyspark.sql.functions import *

def func_url_lengh(var1):
    return len(var1)
func_url_lengh_udf = udf(func_url_lengh, IntegerType())

def func_url_big(var1):
    if (len(var1) > 80):
        return 1
    else: 
        return 0
func_url_big_udf = udf(func_url_big, IntegerType())

### Every URL with More than 80 chars, will be massive .

### From Commom Crawl Import global urls :: Asume as verified=0
#ccrawl_trainpd=pd.read_json('/home/siemanalyst/notebooks/siem/tail-CC-MAIN-2018-43-index-.com.json', typ='series').to_frame('count')
#ccrawl_trainpd.head(100)
#ccrawl_trainds=remove_others(ccrawl_trainpd, {"url"})
#ccrawl_trainds.head(3)

## Get all URL Commom Crawl for Top Level Domains : *.com *.net *.org *.co.uk *.ru 
ccrawl_traindf = sqlContext.read.json("hdfs:///user/siemanalyst/data/raw/commoncrawl/dt=20181112/clean-CC-MAIN-2018-43-index-.*.json")

## Assume URLS as not phishing verified=0
ccrawl_trainds = ccrawl_traindf.withColumn('verified',lit(0)).withColumn('url_length',func_url_lengh_udf(ccrawl_traindf['url'])).withColumn('massiveurl',func_url_big_udf(ccrawl_traindf['url'])).drop('charset','digest','filename','verification_time','languages','length','mime','mime-detected','offset','status')
ccrawl_trainds.printSchema()

ccrawl_train_pd=ccrawl_trainds.toPandas()  
ccrawl_test_pd=ccrawl_trainds.toPandas().head(1000)

ccrawl_train_hf=h2o.H2OFrame(ccrawl_train_pd)
ccrawl_test_hf=h2o.H2OFrame(ccrawl_test_pd)

### From Phishtank Import Malware urls
phishtank_trainpd=pd.read_csv("/home/siemanalyst/notebooks/siem/verified_online.csv")
phishtank_trainds=phishtank_trainpd.drop(columns=['phish_id','phish_detail_url','submission_time','verification_time','online','target'])
phishtank_trainds['url_length'] = phishtank_trainds['url'].map(str).apply(len)
phishtank_trainds['massiveurl'] = np.where(phishtank_trainds['url_length'] > 80 , 1, 0)
#phishtank_trainds.head(6)
phishtank_train=h2o.H2OFrame(phishtank_trainds)

phishtank_testpd=pd.read_csv("/home/siemanalyst/notebooks/siem/verified_online.csv")
phishtank_testds=phishtank_testpd.drop(columns=['phish_id','phish_detail_url','submission_time','verification_time','online','target'])
phishtank_testds['url_length'] = phishtank_testds['url'].map(str).apply(len)
phishtank_testds['massiveurl'] = np.where(phishtank_testds['url_length'] > 80 , 1, 0)
#phishtank_testds.head(6)
phishtank_test=h2o.H2OFrame(phishtank_testds.head(1000))


# Merge the data frames. The result is a single dataset with three columns.
#print(ccrawl_train_hf)
#print(ccrawl_test_hf.describe)
#print(phishtank_train.describe)
#print(phishtank_test.describe)

# Merge all of df2 into df1 with rbind .
# As merge() doesn't work if both H2O/dataframes are the same

train = ccrawl_train_hf.rbind(phishtank_train)
train.describe

test = ccrawl_test_hf.rbind(phishtank_test)
test.describe

# Identify predictors and response
x = train.columns
y = 'massiveurl'
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 40 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=40, seed=1999)
aml.train(x=x, y=y, training_frame=train)


root
 |-- url: string (nullable = true)
 |-- verified: integer (nullable = false)
 |-- url_length: integer (nullable = true)
 |-- massiveurl: integer (nullable = true)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


url,verified,url_length,massiveurl
http://www2.co.uk/musings/2009/11/,0,34,0
http://www2.co.uk/musings/2009/12/,0,34,0
http://www2.co.uk/musings/2010/05/,0,34,0
http://www2.co.uk/musings/2010/08/,0,34,0
http://www2.co.uk/musings/author/admin/,0,39,0
http://www2.co.uk/musings/category/facebook-privacy/,0,52,0
http://www2.co.uk/musings/category/internet-privacy/,0,52,0
http://www2.co.uk/musings/category/internet-secrecy/,0,52,0
http://www2.co.uk/musings/category/uncategorized/,0,49,0
http://www2.co.uk/musings/facebooks-privacy-policy-mandatory/,0,61,0



AutoML progress: |████████████████████████████████████████████

In [3]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,auc,logloss,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_AutoML_20181112_161712,1,0.000134087,0,0.000205558,4.2254e-08
XGBoost_2_AutoML_20181112_161712,1,0.000299954,0,0.000367022,1.34705e-07
XGBoost_1_AutoML_20181112_161712,1,0.000134759,0,0.000164911,2.71956e-08
XRT_1_AutoML_20181112_161712,1,0.00112938,0,0.0131951,0.00017411
StackedEnsemble_AllModels_AutoML_20181112_161712,1,0.00011539,0,0.000160712,2.58282e-08
DRF_1_AutoML_20181112_161712,1,0,0,0,0
GLM_grid_1_AutoML_20181112_161712_model_1,1,0.0233865,0,0.074118,0.00549348
